In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from SALib.analyze import sobol
import warnings
from ema_workbench.analysis import feature_scoring
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    perform_experiments,
    Samplers,
    SequentialEvaluator,
)
from dike_model_function import DikeNetwork
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time
from ema_workbench.analysis import prim
from ema_workbench.em_framework.parameters import Constant
import pickle
warnings.filterwarnings("ignore")

In [2]:
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [10]:
ScenResul = pd.read_csv('data\OpenScenRes.csv').drop(columns="Unnamed: 0")

In [17]:
ScenResul_mean =ScenResul.mean()

In [ ]:
epsilon = []

In [22]:
#load it
with open("data\ScenariosOpenExplo", 'rb') as file2:
    Scenarios = pickle.load(file2)

In [ ]:
from ema_workbench.em_framework.optimization import ArchiveLogger, EpsilonProgress

# we need to store our results for each seed
results = []
convergences = []

with SequentialEvaluator(dike_model) as evaluator:
    # we run again for 5 seeds
    for i in range(2):
        # we create 2 covergence tracker metrics
        # the archive logger writes the archive to disk for every x nfe
        # the epsilon progress tracks during runtime
        convergence_metrics = [
            ArchiveLogger(
                r"C:\Users\roelo\Videos\archive",
                [l.name for l in dike_model.levers],
                [o.name for o in dike_model.outcomes],
                base_filename=f"{i}.tar.gz",
            ),
            
            EpsilonProgress(),
        ]

        result, convergence = evaluator.optimize(
            nfe=10,
            searchover="levers",
            epsilons=[0.1] * len(dike_model.outcomes),
            convergence=convergence_metrics,
            reference = Scenarios[0]
        )

        results.append(result)
        convergences.append(convergence)

  0%|                                                   | 0/10 [00:00<?, ?it/s]

In [ ]:
results[0].drop(columns=["A.1_Expected Number of Deaths","A.1_Expected Annual Damage"],inplace=True)

In [ ]:
results[1].drop(columns=["A.1_Expected Number of Deaths","A.1_Expected Annual Damage"],inplace=True)

In [ ]:
l=result.describe()

In [ ]:
l.loc[l.index.isin(["min","max"])]

In [ ]:
from optimization_tuned import epsilon_nondominated,HypervolumeMetric, Hypervolume

In [ ]:
# from ema_workbench.em_framework.optimization import to_problem
# 
# problem = to_problem(dike_model, searchover="levers")
# 
# reference_set = epsilon_nondominated(results, [0.05] * len(dike_model.outcomes), problem)



In [ ]:
k = r"C:\Users\roelo\Videos\archive"


In [ ]:
all_archives = []

for i in range(2):
    archives = ArchiveLogger.load_archives(k+ "\\" + f"{i}.tar.gz")
    del archives[0]["Unnamed: 0"]
    del archives[100]["Unnamed: 0"]
    
    del archives[0]["A.1_Expected Number of Deaths"]
    del archives[100]["A.1_Expected Number of Deaths"]
    
    del archives[0]["A.1_Expected Annual Damage"]
    del archives[100]["A.1_Expected Annual Damage"]
    all_archives.append(archives)

In [ ]:
all_archives

In [ ]:
dike_model_problem =dike_model
dike_model_problem.outcomes.__delitem__("A.1_Expected Number of Deaths")
dike_model_problem.outcomes.__delitem__("A.1_Expected Annual Damage")

In [ ]:
all_archives

In [ ]:
dike_model_problem

In [ ]:
from optimization_tuned import (
    HypervolumeMetric,
    GenerationalDistanceMetric,
    EpsilonIndicatorMetric,
    InvertedGenerationalDistanceMetric,
    SpacingMetric,
)
from ema_workbench.em_framework.optimization import to_problem

problem = to_problem(dike_model_problem, searchover="levers")

reference_set = epsilon_nondominated(results, [0.1] * (len(dike_model.outcomes)-2), problem)

hv = HypervolumeMetric(reference_set, problem)
print(1)
gd = GenerationalDistanceMetric(reference_set, problem, d=1)
print(2)
ei = EpsilonIndicatorMetric(reference_set, problem)
print(3)
ig = InvertedGenerationalDistanceMetric(reference_set, problem, d=1)
print(4)
sm = SpacingMetric(problem)
print(5)


metrics_by_seed = []
for archives in all_archives:
    metrics = []
    for nfe, archive in archives.items():
        print(nfe)
        scores = {
            # "generational_distance": gd.calculate(archive),
            # "epsilon_indicator": ei.calculate(archive),
            # "inverted_gd": ig.calculate(archive),
            # "spacing": sm.calculate(archive),
            "nfe": int(nfe),
        }
        metrics.append(scores)
    metrics = pd.DataFrame.from_dict(metrics)

    # sort metrics by number of function evaluations
    metrics.sort_values(by="nfe", inplace=True)
    metrics_by_seed.append(metrics)

In [ ]:
metrics_by_seed

In [ ]:
metrics.sort_values(by="nfe", inplace=True)
    metrics_by_seed.append(metrics)

In [ ]:
metrics_by_seed

In [ ]:
sns.set_style("white")
fig, axes = plt.subplots(nrows=6, figsize=(8, 12), sharex=True)

ax1, ax2, ax3, ax4, ax5, ax6 = axes

for metrics, convergence in zip(metrics_by_seed, convergences):
    ax1.plot(metrics.nfe, metrics.hypervolume)
    ax1.set_ylabel("hypervolume")

    ax2.plot(convergence.nfe, convergence.epsilon_progress)
    ax2.set_ylabel("$\epsilon$ progress")

    ax3.plot(metrics.nfe, metrics.generational_distance)
    ax3.set_ylabel("generational distance")

    ax4.plot(metrics.nfe, metrics.epsilon_indicator)
    ax4.set_ylabel("epsilon indicator")

    ax5.plot(metrics.nfe, metrics.inverted_gd)
    ax5.set_ylabel("inverted generational\ndistance")

    ax6.plot(metrics.nfe, metrics.spacing)
    ax6.set_ylabel("spacing")

ax6.set_xlabel("nfe")


sns.despine(fig)

plt.show()